In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plaid.denoisers import UIPA
ckpt_path = "/shared/amyxlu/dprot/ckpts/2eiqqk2u"
denoiser, unmatched_keys = UIPA.load_from_checkpoint(ckpt_path, return_unmatched_keys=True)

In [19]:
from plaid.utils import LatentScaler
from plaid.diffusion.beta_schedulers import CosineBetaScheduler, LinearBetaScheduler, SigmoidBetaScheduler
from plaid.diffusion import GaussianDiffusion
import torch
latent_scaler = LatentScaler(mode="channel_standardize", origin_dataset="uniref")
beta_scheduler = SigmoidBetaScheduler(start=0, end=3, tau=0.5)

diffusion = GaussianDiffusion(
    model=denoiser,
    latent_scaler=latent_scaler,
    beta_scheduler=beta_scheduler,
    sampling_timesteps=500,
)
device = torch.device("cuda:0")
diffusion = diffusion.to(device)

In [12]:
# from plaid.diffusion import GaussianDiffusion, ElucidatedDiffusion
# import torch
# device = torch.device("cuda:0")
# denoiser = denoiser.to(device)

# latent_scaler = LatentScaler(mode="channel_standardize", origin_dataset="cath")

# diffusion = ElucidatedDiffusion(
#     denoiser=denoiser,
#     latent_scaler=latent_scaler,
#     # sigma_data=1.,
#     # sigma_max=100.,
#     # rho=5,
#     # P_std=0.8
# )
# diffusion = diffusion.to(device)


In [21]:
import torch
batch_size = 16
UNCOND_IDX = 6
x = torch.full((batch_size, 3),  UNCOND_IDX).long().to(device)
cond_dict = {"secondary_structure": x}
model_kwargs = {"cond_dict": cond_dict}

sampled = diffusion.sample(
    shape=(16,64,1024),
    clip_denoised=True,
    model_kwargs=model_kwargs,
)

print(sampled.mean(), sampled.std())


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

tensor(-2762.2837, device='cuda:0') tensor(117722.1406, device='cuda:0')


In [22]:
from plaid.callbacks import SampleCallback

sampler = SampleCallback(
    diffusion=diffusion,
    model=denoiser,
    calc_fid=True
)

strs, log_dict = sampler.construct_sequence(sampled, device)
strs

percentage similarty to argmax idx: 1.000
Perplexity: 3.461


['DDDDADDIDDDADDDIADDDDDDDDDDDDDDDDDDADDDDDDDDDADDDDDDDDDDDDDDIDDD',
 'DDDDDADDDDDDDDDDDADDDDDDDDDDADADDDDDDDDDDDDDDDDDADDDDDDDDDDDDDDI',
 'DDDDDDDDDDDDDDDDDDDDDADDDDDDDDDDDDDDDDDDDDDDDDADAAADDDDDDDDDDDDD',
 'DDADDDDDDADADDDIDDDADIDDDDDDDDDDAADADDDDDADDDDADDDADDADDDIDDDDDD',
 'DDDDDDDDDDDDDDDDDDADDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDADDDDD',
 'ADDDDDDDDDADDDDDDDDDDDAADDDDDADDDDDDADDDDDDDDDDDDDDDDDDDDDDDDDDD',
 'DDDDDDDDDDDDDDDDDDDADDADDDDDDDDDADDDDDDDDDADDDDDDIDDDDDADDADDDDD',
 'DDDDDADDDDDDDDDDDDDDDDDDDADADDDDDDDDADDDDDDDDDADDDDDDADDDDDDDDDD',
 'DDDDDDDADDDDDDDDDDDDDDDDDADDDDDDDDDDDDADDDDDDDDDDDDDDDDADDDDDDDD',
 'DDDDDDIDDDDDDDADDDDDDAADDDDDDDDDDADDDDAADADDDADDDDDDDAADDDDDDDDD',
 'DIDDDADDDDDDDADDDDDDDDDDDDADDDDDDDDDADDDDDDDDDDIDDDDDDDDDDDDDDDD',
 'DDDDDDDDDDDDIADDDDDDDDDDDDDDDDIDADDADADADDDDADDDDDAADDAADDDDADDA',
 'ADDDDDDDDDDDDDDDDDDDDDADDDDDDDDDDDAADDDDDDDDDDDDDDDDDDDDDDDDDDDD',
 'DAADDDDDDDDDADDADDDDDDDDDDDDDDDDDDDDADDDADDDDDDDDDDDDDDDDDDDDDDD',
 'DDDDDDDDADDDDADDDDDDDDDDADADDDAD

In [24]:
# from plaid.utils import view_py3Dmol
# view_py3Dmol(pdb_strs[0])